In [332]:
import pandas as pd

In [333]:
class Account:
    month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    season = ["ns", "ns", "ns", "s", "s", "s", "s", "s", "s", "ns", "ns", "ns"]
    user_type = {"la" : "la","lv" : "lv","hv" : "hv"}

    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.contract = self.init_values()
        self.unitcprice = {"s" : self.init_values(),"ns" : self.init_values()}
        self.demand = {}
        self.baseprice = {}
        self.exceed_stage_1 = {}
        self.exceed_stage_2 = {}
        self.exceed_stage_1_price = {}
        self.exceed_stage_2_price = {}
        self.exceed_price = {}
        self.totalprice = {}

    
        
    #Method for input the demand
    #demand_r: regular demand
    #demand_ns = non-summer demand
    #demand_sat = Saturday demand
    #demant_op = off-peak demand


    def init_values(self):
        return {"r" : 0, "hp" :0, "ns": 0, "shp" : 0,"op" : 0}
    
    def set_values(self, dict, values):
        dict = {"r" : values[0], "hp" :values[1], "ns": values[2], "shp" : values[3],"op" : values[4]}
        return dict

    def set_contract(self, values = [0,0,0,0,0]): 
        self.contract = self.set_values(self.contract, values)

    def set_demand(self, month_index, values = [0,0,0,0,0]): 
        self.demand[month_index] = self.set_values(self.demand, values)
        
    def set_unitcontractprice(self, season = "s", values = [0,0,0,0,0]): 
        self.unitcprice[season] = self.set_values(self.unitcprice[season], values)

    def get_contract(self, month_index):
        contract_month = self.contract
        return contract_month

    def get_demand(self, month_index):
        demand_month = self.demand[month_index]
        return demand_month
    
    def get_unitcontractprice(self, month_index):
        s = self.season[month_index]
        unitcprice_month = self.unitcprice[s]
        return unitcprice_month
        
    def caculate_exceed(self, contract, demand):
        cacuEx = lambda x, y : x-y if x-y > 0 else 0 
        exceed = {
            "r" : cacuEx(demand["r"],contract["r"]),
            "hp" :cacuEx(demand["hp"],contract["r"]+contract["hp"]),
            "ns" : cacuEx(demand["ns"],contract["r"]+contract["ns"]),
            "shp" : cacuEx(demand["shp"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]),
            "op" : cacuEx(demand["op"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]+contract["op"])
            }
        return exceed
    
    
    def calculate_baseprice(self, contract, unitcprice):
        common = contract["shp"]+contract["op"]-(contract["r"]+contract["ns"])*0.5
        return contract["r"]*unitcprice["r"]+contract["ns"]*unitcprice["ns"]+contract["hp"]*unitcprice["hp"]+contract["shp"]*common
    

    def caculate_exceed_stage(self, contract, exceed):
        caculate_stage_1 = lambda x, y: 0.1*y if x >= 0.1*y else x
        caculate_stage_2 = lambda x, y: max(x - y,0)

        hp = max(exceed["hp"]-exceed["r"], 0)
        shp = max(exceed["shp"]-max(exceed["r"], exceed["hp"]), 0)
        op = max(exceed["op"]-max(exceed["r"],exceed["hp"], exceed["shp"]), 0)

        exceed_stage_1 = {
            "r" : caculate_stage_1(exceed["r"], contract["r"]),
            "hp": caculate_stage_1(hp, contract["hp"]),
            "ns": caculate_stage_1(exceed["ns"], contract["ns"]),
            "shp": caculate_stage_1(shp, contract["shp"]),
            "op": caculate_stage_1(op, contract["op"])
        }

        exceed_stage_2 = {
            "r" : caculate_stage_2(exceed["r"],  exceed_stage_1["r"]),
            "hp" : caculate_stage_2(hp, exceed_stage_1["hp"]),
            "ns" : caculate_stage_2(exceed["ns"], exceed_stage_1["ns"]),
            "shp" : caculate_stage_2(shp, exceed_stage_1["shp"]),
            "op" : caculate_stage_2(op, exceed_stage_1["op"]) 
        }

        exceed_stages = {
            "stage_1" : exceed_stage_1,
            "stage_2" : exceed_stage_2
        }

        return exceed_stages
    
    def caculate_exceed_price(self, exceed_stage, unitcprice, rate = 1):
        exceed_price = {
            "r" : exceed_stage["r"]*unitcprice["r"]*rate,
            "hp" : exceed_stage["hp"]*unitcprice["hp"]*rate,
            "ns" : exceed_stage["ns"]*unitcprice["ns"]*rate,
            "shp" : exceed_stage["shp"]*unitcprice["shp"]*rate,
            "op" : exceed_stage["op"]*unitcprice["op"]*rate
        }

        return  exceed_price

    def caculate_total(self):
        for month_index in range(12):
            contract = self.get_contract(month_index)
            demand = self.get_demand(month_index)
            exceed = self.caculate_exceed(contract, demand)
            exceed_stage_1 = self.caculate_exceed_stage(contract, exceed)["stage_1"]
            exceed_stage_2 = self.caculate_exceed_stage(contract, exceed)["stage_2"]
            unitcprice = self.get_unitcontractprice(month_index)
            baseprice = self.calculate_baseprice(contract, unitcprice)

            exceed_price = {
                "stage_1" : self.caculate_exceed_price(exceed_stage_1, unitcprice, 2),
                "stage_2" : self.caculate_exceed_price(exceed_stage_2, unitcprice, 3)
                }
            
            exceed_stage_price = {
                "stage_1" : sum(exceed_price["stage_1"].values()),
                "stage_2" : sum(exceed_price["stage_2"].values())
                }

            self.baseprice[month_index] = baseprice
            self.exceed_stage_1[month_index] = exceed_stage_1
            self.exceed_stage_2[month_index] = exceed_stage_2
            self.exceed_stage_1_price[month_index] = exceed_stage_price["stage_1"]
            self.exceed_stage_2_price[month_index] = exceed_stage_price["stage_2"]
            self.exceed_price[month_index] = exceed_stage_price["stage_1"] + exceed_stage_price["stage_2"]
            self.totalprice[month_index] = baseprice +  exceed_stage_price["stage_1"] +  exceed_stage_price["stage_2"]


In [334]:
user1 = Account(1000,'C1')
user1.set_contract([300,0,0,0,0])
user1.set_unitcontractprice("s", [223.6,0,166.9,44.7,44.7])
user1.set_unitcontractprice("ns", [166.9,0,166.9,33.3,33.3])
demand = [[68,0,0,62,36],
          [66,0,0,42,37],
          [177,0,0,154,36],
          [278,0,0,179,150],
          [276,0,0,164,39],
          [309,0,0,201,40],
          [357,0,0,215,48],
          [329,0,0,205,174],
          [342,0,0,312,54],
          [326,0,0,234,47],
          [240,0,0,173,92],
          [157,0,0,153,36]]

for month_index in range(12):
   s = user1.season[month_index]
   user1.set_demand(month_index, demand[month_index])

In [335]:
#return the total price of each month
user1.caculate_total()

# create a dataframe to store the contract, demand, unit contract price, base price, exceed stage 1, exceed stage 2, exceed stage 1 price, exceed stage 2 price, exceed price, total price

for month_index in range(12):
    month = user1.month[month_index]
    contract = user1.get_contract(month_index)
    demand = user1.get_demand(month_index)
    unitcprice = user1.get_unitcontractprice(month_index)
    baseprice = user1.baseprice[month_index]
    exceed_stage_1 = user1.exceed_stage_1[month_index]
    exceed_stage_2 = user1.exceed_stage_2[month_index]
    exceed_stage_1_price = user1.exceed_stage_1_price[month_index]
    exceed_stage_2_price = user1.exceed_stage_2_price[month_index]
    exceed_price = user1.exceed_price[month_index]
    totalprice = user1.totalprice[month_index]
    print("month: ", month)
    print("contract: ", contract)
    print("demand: ", demand)
    print("unit contract price: ", unitcprice)
    print("base price: ", baseprice)
    print("exceed stage 1: ", exceed_stage_1)
    print("exceed stage 2: ", exceed_stage_2)
    print("exceed stage 1 price: ", exceed_stage_1_price)
    print("exceed stage 2 price: ", exceed_stage_2_price)
    print("exceed price: ", exceed_price)
    print("total price: ", totalprice)
    print("")


month:  Jan
contract:  {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
demand:  {'r': 68, 'hp': 0, 'ns': 0, 'shp': 62, 'op': 36}
unit contract price:  {'r': 166.9, 'hp': 0, 'ns': 166.9, 'shp': 33.3, 'op': 33.3}
base price:  50070.0
exceed stage 1:  {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}
exceed stage 2:  {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}
exceed stage 1 price:  0.0
exceed stage 2 price:  0.0
exceed price:  0.0
total price:  50070.0

month:  Feb
contract:  {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
demand:  {'r': 66, 'hp': 0, 'ns': 0, 'shp': 42, 'op': 37}
unit contract price:  {'r': 166.9, 'hp': 0, 'ns': 166.9, 'shp': 33.3, 'op': 33.3}
base price:  50070.0
exceed stage 1:  {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}
exceed stage 2:  {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}
exceed stage 1 price:  0.0
exceed stage 2 price:  0.0
exceed price:  0.0
total price:  50070.0

month:  Mar
contract:  {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0

In [336]:
# create a function to choose the best contract for each month
user1 = Account(1000,'C1')
user1.set_contract("s", [300,0,0,0,0])
user1.set_contract("ns", [300,0,0,0,0])

for month_index in range(12):
   s = user1.season[month_index]
   user1.set_demand(month_index, demand[month_index])

def chooseBestContract(user):
   current_contract = user.contract
   demand_pool = {
         "r" : [,0],
         "hp" : [0,0],
         "ns" : [0,0],
         "shp" : [0,0],
         "op" : [0,0]
   }
   
   c_r = random.randint(0, 500)
   c_hp = random.randint(0, 500)
   c_ns = random.randint(0, 500)
   c_shp = random.randint(0, 500)
   c_op = random.randint(0, 500)
   c = [c_r, c_hp, c_ns, c_shp, c_op]

   user1.set_contract("s", c)
   user1.set_contract("ns", c)
    






   totalprice = user1.totalprice

    


TypeError: Account.set_contract() takes from 1 to 2 positional arguments but 3 were given